0. Imports

In [ ]:
import pandas as pd

1. Load csv

In [ ]:
# change to your file location
df = pd.read_csv('/content/drive/MyDrive/Škola/DM/spravanie_studentov_v_zavislosti_od_casu_II/MLM_STUD_stats.csv', ';', usecols=range(0,10))
df.head(10)

,Effect,Intercept,HODINA,HODINA_STV,PO,UT,STR,STVR,PIA,SO
0,uvod_STUD,1.34835,0.34182,-0.01563,-0.63353,1.10307,0.11376,-0.49911,0.68682,0
1,studium_STUD,-2.84924,0.38448,-0.00867,0.95518,3.02822,1.54827,1.22148,1.20767,0
2,oznamy_STUD,1.51459,-0.38342,0.01248,1.65098,3.04913,1.59056,0.14253,-1.24192,0


2. Create collection of weekdays

In [ ]:
days = ['PO', 'UT', 'STR', 'STVR', 'PIA', 'SO']

3. Create logits estimate

In [ ]:
dataframe_collection = {} 
index = 0

# Cycle through weekdays
for day in days:

      # Create empty dataframe for weekday
      df_day = pd.DataFrame()

      # Cycle through hours from 7 to 23
      for x in range (7,23):

        # Create logits estimates
        logit_uvod = df.at[index, 'Intercept'] + df.at[index, 'HODINA']*x+df.at[index, 'HODINA_STV']*(x*x)+df.at[index, day]
        logit_studium = df.at[index+1, 'Intercept'] + df.at[index+1, 'HODINA']*x+df.at[index+1, 'HODINA_STV']*(x*x)+df.at[index+1, day]
        logit_oznamy = df.at[index+2, 'Intercept'] + df.at[index+2, 'HODINA']*x+df.at[index+2, 'HODINA_STV']*(x*x)+df.at[index+2, day]

        # Create new row and append it to dataframe
        new_row = {'0_hod': x, '1_uvod': logit_uvod, '2_studium':logit_studium, '3_oznamy':logit_oznamy}
        df_day = df_day.append(new_row, sort=False, ignore_index=True)

      # add dataframe to dictionary of dataframes
      dataframe_collection[day] = df_day

4. Export to excel

In [ ]:
# Creating Excel Writer Object from Pandas  
writer = pd.ExcelWriter('STUD_logits.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('STUD')
writer.sheets['STUD'] = worksheet

i = 0
for day in days:
  worksheet.write(0, i, str(day + " - ODHAD LOGITOV"))
  dataframe_collection[day].to_excel(writer, sheet_name='STUD',startrow=1 , startcol=i, index=False)
  i = i+5
writer.save()